In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastrl['dev']  # upgrade fastrl on colab

In [ ]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON

In [ ]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from torch.utils.data import Dataset
from torch import nn
import torch


import numpy as np
import gym
import time,sys
import torch.multiprocessing as mp
import pandas as pd

# Local modules

In [ ]:
# export
class ExperienceSource(Stateful):
    _stateattrs=('pool',)
    def __init__(self,env:str,agent,n_envs:int=1,steps_count:int=1,steps_delta:int=1,
                 vectorized:bool=False,seed:int=None,**kwargs):
        store_attr()
        self.reset_times=0
        super().__init__(**kwargs)
            
    def _init_state(self):
        print('reseting')
        self.reset_times+=1
        self.pool:List[gym.Env]=[gym.make(self.env) for _ in range(self.n_envs)]
             
    def __iter__(self):
        while True: yield self.reset_times

In [ ]:
class TestDataset(IterableDataset):
    def __init__(self,source=None): store_attr('source')
    def __iter__(self): return iter(self.source)
    def wif(self):
        print('hi')
        dataset.source._init_state()

In [ ]:
source=ExperienceSource('CartPole-v1',None)
dataset=TestDataset(source)

reseting


In [ ]:
def worker_init_fn():
    worker_info = torch.utils.data.get_worker_info()
    print(worker_info,worker_info.dataset.dataset.source)
    dataset = worker_info.dataset  # the dataset copy in this worker process
    if hasattr(dataset,'source') and isinstance(dataset.source,Stateful):
        print('reseting')
        dataset.source._init_state()

In [ ]:
for x in DataLoader(dataset,num_workers=3,n=10,persistent_workers=True,wif=worker_init_fn):
    print(x)

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 146, in _worker_loop
    init_fn(worker_id)
  File "/opt/conda/lib/python3.7/site-packages/fastai/data/load.py", line 17, in _wif
    ds.wif()
  File "<ipython-input-7-548bc5f71775>", line 3, in worker_init_fn
    print(worker_info,worker_info.dataset.dataset.source)
AttributeError: '_FakeLoader' object has no attribute 'source'
